## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-21-43-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
1,2025-07-09-21-43-07 +0000,nyt,"Henry Mount Charles, Whose Castle Was a Mecca ...",https://www.nytimes.com/2025/07/09/world/europ...
2,2025-07-09-21-41-44 +0000,nyt,Trump Tariffs Aim to Settle Scores With Countr...,https://www.nytimes.com/2025/07/09/business/ec...
3,2025-07-09-21-39-44 +0000,nyt,Russia Makes Record Attack on Ukraine as Trump...,https://www.nytimes.com/2025/07/09/world/europ...
4,2025-07-09-21-34-15 +0000,nyt,Yemen’s Houthi Militia Took Sailors Hostage Af...,https://www.nytimes.com/2025/07/09/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2299/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,21
2,tariffs,6
29,attack,5
30,ukraine,5
62,israel,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2025-07-09-21-39-44 +0000,nyt,Russia Makes Record Attack on Ukraine as Trump...,https://www.nytimes.com/2025/07/09/world/europ...,44
0,2025-07-09-21-43-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,41
45,2025-07-09-14-50-42 +0000,bbc,Ukraine suffers heaviest attack as Trump criti...,https://www.bbc.com/news/articles/c2k19q4j07zo,39
46,2025-07-09-14-41-08 +0000,nyt,"Putin Escalates His War Against Ukraine, Undet...",https://www.nytimes.com/2025/07/09/world/europ...,38
2,2025-07-09-21-41-44 +0000,nyt,Trump Tariffs Aim to Settle Scores With Countr...,https://www.nytimes.com/2025/07/09/business/ec...,37


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,44,2025-07-09-21-39-44 +0000,nyt,Russia Makes Record Attack on Ukraine as Trump...,https://www.nytimes.com/2025/07/09/world/europ...
21,24,2025-07-09-19-48-23 +0000,nyt,Former President Yoon Suk Yeol of South Korea ...,https://www.nytimes.com/2025/07/09/world/asia/...
0,20,2025-07-09-21-43-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
50,19,2025-07-09-13-51-41 +0000,bbc,French police raid on National Rally HQ prompt...,https://www.bbc.com/news/articles/ckg5kd04e1jo
37,18,2025-07-09-16-28-07 +0000,nyt,Israel’s Deadly Assault on Iran Prison Incites...,https://www.nytimes.com/2025/07/06/world/middl...
64,16,2025-07-09-10-55-24 +0000,nyt,Jailed Kurdish Leader Declares the End to PKK’...,https://www.nytimes.com/2025/07/09/world/middl...
38,15,2025-07-09-16-06-35 +0000,bbc,EU hopes to agree US tariff deal 'in coming days',https://www.bbc.com/news/articles/cev0n01y8wwo
90,14,2025-07-09-00-11-42 +0000,bbc,Moment house is swept away in New Mexico flash...,https://www.bbc.com/news/videos/cren2zq1y3qo
13,13,2025-07-09-20-36-58 +0000,bbc,'Huge wall of water': Texas man stood on meter...,https://www.bbc.com/news/articles/c3358pz41d6o
78,13,2025-07-09-06-21-45 +0000,bbc,Seoul returns six North Koreans with 'strong d...,https://www.bbc.com/news/articles/cwyrwp8r1xgo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
